<a href="https://colab.research.google.com/github/OJB-Quantum/Notebooks-for-Ideas/blob/main/Detect_Red_Pixels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Authored by Onri Jay Benally (2025)

Open Access (CC-BY-4.0)

In [ ]:
"""Detect red pixels/colors in an uploaded image.

Supported formats
-----------------
* PNG
* JPEG / JPG
* SVG (vector)

The script runs in Google Colab and uses the standard file-upload
widget. For raster images the hue channel of the HSV color space is
used to decide whether a pixel belongs to the “red” sector. For SVG
files color specifications are parsed and the same hue test is applied.
"""

# ----------------------------------------------------------------------
# Standard-library imports
# ----------------------------------------------------------------------
import os
import re
import xml.etree.ElementTree as ET
from typing import List, Optional, Tuple

# ----------------------------------------------------------------------
# Third-party imports
# ----------------------------------------------------------------------
import numpy as np
from google.colab import files
from PIL import Image
import matplotlib.colors as mcolors

# ----------------------------------------------------------------------
# Helper functions
# ----------------------------------------------------------------------


def upload_file() -> str:
    """Prompt the user to upload a file and return its path.

    Returns
    -------
    str
        Path of the uploaded file in the Colab environment.
    """
    uploaded = files.upload()
    if not uploaded:
        raise RuntimeError("No file was uploaded.")
    # ``files.upload`` returns a dict {filename: bytes}
    filename = next(iter(uploaded))
    # Write the uploaded bytes to a file so other libraries can read it.
    with open(filename, "wb") as f:
        f.write(uploaded[filename])
    return filename


def load_raster_image(path: str) -> np.ndarray:
    """Load a PNG or JPEG image and return an RGB array.

    Parameters
    ----------
    path : str
        Path to the image file.

    Returns
    -------
    np.ndarray
        Float array with shape (H, W, 3) and values in the range [0, 1].
    """
    with Image.open(path) as img:
        rgb = img.convert("RGB")
        array = np.asarray(rgb, dtype=np.float32) / 255.0
    return array


def compute_hsv(image: np.ndarray) -> np.ndarray:
    """Convert an RGB image to HSV.

    Parameters
    ----------
    image : np.ndarray
        RGB image where each channel is in [0, 1].

    Returns
    -------
    np.ndarray
        HSV image with the same shape as the input.
    """
    return mcolors.rgb_to_hsv(image)


def mask_red_pixels(hsv: np.ndarray) -> np.ndarray:
    """Create a boolean mask that selects red pixels.

    A pixel is considered red when its hue lies inside the red sector
    (0-20° or 340-360°) of the HSV color space. The hue channel is
    stored in the range [0, 1].

    Parameters
    ----------
    hsv : np.ndarray
        HSV image where the last dimension holds (H, S, V).

    Returns
    -------
    np.ndarray
        Boolean mask of shape (H, W); ``True`` marks a red pixel.
    """
    hue = hsv[..., 0]
    # Red occupies the lower 5% and the upper 5% of the hue wheel.
    low = hue <= 0.0556   # ≈ 20 / 360
    high = hue >= 0.9444  # ≈ 340 / 360
    return np.logical_or(low, high)


def analyze_red_saturation(saturations: np.ndarray) -> Tuple[int, float, float, float]:
    """Return simple statistics for a collection of saturation values.

    Parameters
    ----------
    saturations : np.ndarray
        1-D array of saturation values belonging to red pixels/colors.

    Returns
    -------
    tuple
        (pixel_count, min_sat, max_sat, mean_sat).
    """
    count = saturations.size
    if count == 0:
        return 0, 0.0, 0.0, 0.0
    return (
        count,
        float(saturations.min()),
        float(saturations.max()),
        float(saturations.mean()),
    )


# ----------------------------------------------------------------------
# Raster (PNG / JPEG) handling
# ----------------------------------------------------------------------


def scan_raster_file(path: str) -> None:
    """Detect red pixels in a raster image and print saturation statistics.

    Parameters
    ----------
    path : str
        Path to a PNG or JPEG file.
    """
    rgb = load_raster_image(path)
    hsv = compute_hsv(rgb)
    red_mask = mask_red_pixels(hsv)
    red_saturations = hsv[..., 1][red_mask]

    count, min_sat, max_sat, mean_sat = analyze_red_saturation(red_saturations)

    if count == 0:
        print("No red pixels were found.")
        return

    print(f"Red pixels detected: {count}")
    print(
        f"Saturation – min: {min_sat:.3f}, max: {max_sat:.3f}, "
        f"mean: {mean_sat:.3f}"
    )


# ----------------------------------------------------------------------
# SVG handling – color parsing utilities
# ----------------------------------------------------------------------


def _hex_to_rgb(hex_str: str) -> Optional[Tuple[float, float, float]]:
    """Convert a hex color (``#RGB`` or ``#RRGGBB``) to a normalized RGB tuple.

    Parameters
    ----------
    hex_str : str
        Hex color string including the leading ``#``.

    Returns
    -------
    tuple or None
        (R, G, B) in [0, 1] or ``None`` if parsing fails.
    """
    hex_str = hex_str.lstrip("#")
    if len(hex_str) == 3:
        hex_str = "".join(ch * 2 for ch in hex_str)
    if len(hex_str) != 6:
        return None
    try:
        r = int(hex_str[0:2], 16) / 255.0
        g = int(hex_str[2:4], 16) / 255.0
        b = int(hex_str[4:6], 16) / 255.0
        return r, g, b
    except ValueError:
        return None


def _extract_color(value: str) -> Optional[Tuple[float, float, float]]:
    """Resolve a CSS color specification to an RGB tuple.

    Recognized forms
    ----------------
    * ``#RRGGBB`` or ``#RGB``
    * ``rgb(r,g,b)`` / ``rgba(r,g,b,…)``
    * Named CSS colors (e.g. ``red``) – resolved via *matplotlib*.

    Parameters
    ----------
    value : str
        Color string to parse.

    Returns
    -------
    tuple or None
        Normalized (R, G, B) values or ``None`` on failure.
    """
    value = value.strip().lower()

    # Hex literal.
    if value.startswith("#"):
        return _hex_to_rgb(value)

    # rgb() / rgba().
    rgb_match = re.fullmatch(
        r"rgba?\(\s*([0-9]{1,3})\s*,\s*([0-9]{1,3})\s*,\s*([0-9]{1,3})",
        value,
        flags=re.IGNORECASE,
    )
    if rgb_match:
        r, g, b = (int(c) / 255.0 for c in rgb_match.groups())
        return r, g, b

    # Fallback to matplotlib's color parser.
    try:
        return mcolors.to_rgb(value)
    except ValueError:
        return None


def _is_red_hue(rgb: Tuple[float, float, float]) -> Optional[float]:
    """Return the saturation if the color lies in the red hue range.

    Parameters
    ----------
    rgb : tuple
        Normalized (R, G, B) values.

    Returns
    -------
    float or None
        Saturation if the hue is red, otherwise ``None``.
    """
    h, s, _ = mcolors.rgb_to_hsv(rgb)
    if h <= 0.0556 or h >= 0.9444:
        return s
    return None


def scan_svg_file(path: str) -> None:
    """Detect red colors in an SVG file and report their saturations.

    Parameters
    ----------
    path : str
        Path to an SVG file.
    """
    tree = ET.parse(path)
    root = tree.getroot()
    red_saturations: List[float] = []

    for element in root.iter():
        # Direct color attributes.
        for attr in ("fill", "stroke"):
            color_val = element.get(attr)
            if color_val is None:
                continue
            rgb = _extract_color(color_val)
            if rgb is None:
                continue
            sat = _is_red_hue(rgb)
            if sat is not None:
                red_saturations.append(sat)

        # Colors may also be embedded in a ``style`` attribute.
        style_val = element.get("style")
        if not style_val:
            continue
        for part in style_val.split(";"):
            if not part:
                continue
            name, _, val = part.partition(":")
            name = name.strip()
            val = val.strip()
            if name not in ("fill", "stroke"):
                continue
            rgb = _extract_color(val)
            if rgb is None:
                continue
            sat = _is_red_hue(rgb)
            if sat is not None:
                red_saturations.append(sat)

    if not red_saturations:
        print("No red colors were found in the SVG file.")
        return

    sat_array = np.array(red_saturations)
    count, min_sat, max_sat, mean_sat = analyze_red_saturation(sat_array)

    print(f"Red colors detected: {count}")
    print(
        f"Saturation – min: {min_sat:.3f}, max: {max_sat:.3f}, "
        f"mean: {mean_sat:.3f}"
    )


# ----------------------------------------------------------------------
# Main entry point
# ----------------------------------------------------------------------


def main() -> None:
    """Run the upload-and-scan workflow.

    The function determines the file type from its extension and
    dispatches to the appropriate analyzer.
    """
    path = upload_file()
    ext = os.path.splitext(path)[1].lower()

    if ext in (".png", ".jpg", ".jpeg"):
        scan_raster_file(path)
    elif ext == ".svg":
        scan_svg_file(path)
    else:
        print(f"Unsupported file type: {ext}")


if __name__ == "__main__":
    main()